# Model Deployment:

Today we will be going over an easier way to get your models into production using Render. Render is used by most of the Fast.AI students to get their models up, and we can run the files locally ourselves to visually see it. To do this, I have provided download links to all of the standard models Fast.AI have used. 

We will explore productioninzing the following models:
* **Computer Vision:** Cats vs Dogs
* **Tabular:** > or <= $50k
* **NLP:** IMDB

![alt text](https://course.fast.ai/images/render/render-logo.svg)

To use the Render template, you will need docker installed. You can run the following below to get it running locally and test your changes.

In [0]:
docker build -t fastai-v3 . && docker run --rm -it -p 5000:5000 fastai-v3

**OR** Simply run `python3 app/server.py serve`.

Before you do this though, do a `pip3 install -r requirements.txt`

In [0]:
* Or simply run 

## Repo

Jeremy has a render example repo that we will be working off of [here](https://github.com/render-examples/fastai-v3). First thing we need is our models. Let's look at the Pets notebook first!

## Cats vs Dogs

### Training

This should take ~5 minutes to run on your own. For todays purposes though we *just* want the models.

In [0]:
from fastai.vision import *
path = untar_data(URLs.PETS)
path_img = path/'images'
fnames = get_image_files(path_img)
np.random.seed(2)
pat = re.compile(r'/([^/]+)_\d+.jpg$')
data = ImageDataBunch.from_name_re(path_img, fnames, pat, ds_tfms=get_transforms(), size=224)
data = data.normalize(imagenet_stats);
learn = create_cnn(data, models.resnet34, pretrained=True, metrics=error_rate)
learn.fit_one_cycle(2);
learn.unfreeze()
learn.fit_one_cycle(1)

### Exporting the Model

In [0]:
learn.path

PosixPath('/root/.fastai/data/oxford-iiit-pet/images')

Well, that's not very easy to get to. Let's change that to Colab's root working directory!

In [0]:
learn.path = Path('')

Now we can run `learn.export()` and get a pkl file with everything we need! (This will also change where learn.save() points to as well)

In [0]:
learn.export('pets.pkl')

**NOTE** When we want to load the model, we can now run `load_learner(path, fname)`. This is **different** than a simple `learn.save()` and `learn.load()` combination.

### Downloading the model

To download my already-built model, run the following

In [0]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1oIn2_DxTJIYBWofQFOQQFcyO8yxJ3H8G' -O 'pets.pkl'

### Modifying the files we need to

Most of what we will be modifying is the 'server.py' file, which is inside the `app` directory. Now first, I recommend either uploading that file to your own google drive, or to dropbox. Then (if you use google drive for your model) change your sharing settings to 'Public', copy the share URL, and find the 'ID=' section. Copy that bit, and replace 'YOURID' in the below:

'https://docs.google.com/uc?export=download&id=YOURID'


Now let's look at the server.py. I'll post the whole thing below:

In [0]:
import aiohttp
import asyncio
import uvicorn
from fastai import *
from fastai.vision import *
from io import BytesIO
from starlette.applications import Starlette
from starlette.middleware.cors import CORSMiddleware
from starlette.responses import HTMLResponse, JSONResponse
from starlette.staticfiles import StaticFiles

export_file_url = 'https://www.dropbox.com/s/6bgq8t6yextloqp/export.pkl?raw=1'
export_file_name = 'export.pkl'

classes = ['black', 'grizzly', 'teddys']
path = Path(__file__).parent

app = Starlette()
app.add_middleware(CORSMiddleware, allow_origins=['*'], allow_headers=['X-Requested-With', 'Content-Type'])
app.mount('/static', StaticFiles(directory='app/static'))


async def download_file(url, dest):
    if dest.exists(): return
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            data = await response.read()
            with open(dest, 'wb') as f:
                f.write(data)


async def setup_learner():
    await download_file(export_file_url, path / export_file_name)
    try:
        learn = load_learner(path, export_file_name)
        return learn
    except RuntimeError as e:
        if len(e.args) > 0 and 'CPU-only machine' in e.args[0]:
            print(e)
            message = "\n\nThis model was trained with an old version of fastai and will not work in a CPU environment.\n\nPlease update the fastai library in your training environment and export your model again.\n\nSee instructions for 'Returning to work' at https://course.fast.ai."
            raise RuntimeError(message)
        else:
            raise


loop = asyncio.get_event_loop()
tasks = [asyncio.ensure_future(setup_learner())]
learn = loop.run_until_complete(asyncio.gather(*tasks))[0]
loop.close()


@app.route('/')
async def homepage(request):
    html_file = path / 'view' / 'index.html'
    return HTMLResponse(html_file.open().read())


@app.route('/analyze', methods=['POST'])
async def analyze(request):
    return JSONResponse({'result': str(prediction)})


if __name__ == '__main__':
    if 'serve' in sys.argv:
        uvicorn.run(app=app, host='0.0.0.0', port=5000, log_level="info")

Wow! Lot's to unpack here! Let's go through it bit by bit.

For our app to work, we need the following libraries:

In [0]:
import aiohttp
import asyncio
import uvicorn
from fastai import *
from fastai.vision import *
from io import BytesIO
from starlette.applications import Starlette
from starlette.middleware.cors import CORSMiddleware
from starlette.responses import HTMLResponse, JSONResponse
from starlette.staticfiles import StaticFiles

* [aiohttp](https://aiohttp.readthedocs.io/en/stable/): HTTP client/server for asyncio
* [asyncio](https://docs.python.org/3/library/asyncio.html): A python framework for good IO performances
* fastai - We *need* the library for our model
* [starlette](https://www.starlette.io/): "A lightweight ASGI framework for high performance asyncio services"

### The *non* functions:

By non-functions I mean the base variables. As you can see, we have a few things. First, we need a link to download our model, then some filename to give it, and lastly what classes we expect things to fall into.

In [0]:
export_file_url = 'https://www.dropbox.com/s/6bgq8t6yextloqp/export.pkl?raw=1'
export_file_name = 'export.pkl'

classes = ['Abyssinian', 'Bengal', 'Birman', 'Bombay', 'British_Shorthair',
 'Egyptian_Mau', 'Maine_Coon', 'Persian', 'Ragdoll', 'Russian_Blue',
 'Siamese', 'Sphynx', 'american_bulldog', 'american_pit_bull_terrier',
 'basset_hound', 'beagle', 'boxer', 'chihuahua', 'english_cocker_spaniel',
 'english_setter', 'german_shorthaired', 'great_pyrenees', 'havanese',
 'japanese_chin', 'keeshond', 'leonberger', 'miniature_pinscher',
 'newfoundland', 'pomeranian', 'pug', 'saint_bernard', 'samoyed',
 'scottish_terrier', 'shiba_inu', 'staffordshire_bull_terrier', 'wheaten_terrier',
 'yorkshire_terrier']

### The Functions:

We have a few functions, 'analyze', 'homepage', 'download_files', and 'setup_learner'. They all do pretty much what we need to get our model' up and running. Analyze we can adjust to however we want our input data to come in. In our case, we expect the user to upload an image file, ad we extract those bytes, utilize the `open_image` function, and run `learn.predict()`. This will be one of the main functions we will change for our use cases.

Now what is an app_route? Those mean seperate web-pages on our server.

#### analyze

In [0]:
@app.route('/analyze', methods=['POST'])
async def analyze(request):
    img_data = await request.form()
    img_bytes = await (img_data['file'].read())
    img = open_image(BytesIO(img_bytes))
    prediction = learn.predict(img)[0]
    return JSONResponse({'result': str(prediction)})

#### setup_learner and download_file

We don't really need to mess with these as they do exactly what we need, regardless of the model, and they're pretty self-explanitory.

## Running our server!

To run your server locally, navigate to your directory you cloned and run the following **if** you have Docker installed:

`docker build -t fastai-v3 . && docker run --rm -it -p 5000:5000 fastai-v3`

Else run: `python app/serve.py serve`

Now we're running on http://localhost:5000/ !

Try it! Currently the HTML page is set up for Jeremy's lesson 2 where he built a bear classifier, but in our lesson today I won't go over HTML and web-page development, you guys can get that experience on your own.

# Other Model Types

The next one we will look at is NLP-based models. I made an app for CodeFest last year called 'Suspecto' where we utilized an NLP model to help grade a models reliability. Using the standard Fast.AI approach we won first place. We'll borrow some of the website code for that setup today.

## What do we change?

First, change the imports to use whatever libraries you used. In my case, this involved the `fastai.text` library.

Next, we changed analyze. Here, we wanted our model to take the results from `learn.predict()`, and based on a formula we had made as a reliability score, report this back to the user.

In [0]:
@app.route('/analyze', methods=['POST'])
async def analyze(request):
    data = await request.form()
    content = data['content']
    prediction = learn.predict(content)[2]
    reliability = prediction[7]-(prediction[6]*prediction[0]) - prediction[0] - prediction[4] - prediction[5] - prediction[8] - (prediction[1]-prediction[11])
    ReliabilityScore = ((reliability.item())*50)+50
    ReliabilityScore = int(ReliabilityScore)
    return JSONResponse({'result': ReliabilityScore})

Here, notice instead we get ['content'] instead. This is due to our request form changing to a text box. To do this, we adjusted index.html. You see we have a special 'content' checker? This lives inside:

In [0]:
<div class='content'>
    <textarea id="content-upload" cols="85" rows="15"></textarea>
    <div class='analyze'>
        <button id='analyze-button' class='analyze-button' type='button' onclick='analyze()'>Analyze</button>
    </div>
    <div class='result-label'>
        <label id='result-label'></label>
    </div>
</div>

So depending on what you want, see what the HTML equivalent is, and so long as you put it in, you can pass it to our analyze function! Now in our case, we want to instead do a movie review, so let's modify a few things!

In [0]:
@app.route('/analyze', methods=['POST'])
async def analyze(request):
    data = await request.form()
    content = data['content']
    prediction = learn.predict(content)[0]
    return JSONResponse({'Review Rating': str(prediction)})

Looks pretty close to what we had before, doesn't it! 

# Advanced Ideas and Tabular Production



In the *real* world, we need to ask a few questions. How will I get my data? How will it be easiest for my customer to send me their data? For example. Take the image problem. If I am dealing with large sets of data they want me to analyze, I'm not going to expect my customer to sit there and upload 100's of files one by one and then we run them! That would be absurd. Instead, we can tell them to provide us with links to the relevant image websites they want us to run predictions with in the form of a CSV, *or* upload a zip document that contains all of their images into a nice folder structure that we can specify.


In [0]:
import zipfile
import csv

@app.route('/analyze', methods=['POST'])
async def analyze(request):
    data = await request.form()
    content = data['content']
    zip_ref = zipfile.ZipFile(content, 'r')
    mkdir('Downloaded_Images')
    zip_ref.extractall('Downloaded_Images')
    zip_ref.close()
    path2 = Path('Downloaded_Images')
    data = ImageList.from_folder(path)
    learn = load_learner(path, export_file_name, test=data)
    y, _ = learn.get_preds(DatasetType.Test)
    y = torch.argmax(y, dim=1)
    preds = [learn.data.classes[int(x)] for x in y]
    rm -r 'Downloaded_Images'
    resultsFile = open('results.csv', 'wb')
    wr = csv.writer(resultsFile)
    wr.writerows([preds])
    return FileResponse('results.csv')

Now, lets parse this CSV when uploaded and download all of our images. We're going to use the `download_images()` function back from lesson 2 to help with this

In [0]:
import csv
import StringIO

@app.route('/analyze', methods=['POST'])
async def analyze(request):
    data = await request.form()
    content = await (data['file'].read())
    s = str(content, 'utf-8')
    data = StringIO(s)
    !mkdir('Downloaded_Images')
    download_images(data, 'Downloaded_Images')
    path2 = Path('Downloaded_Images')
    data = ImageList.from_folder(path)
    learn = load_learner(path, export_file_name, test=data)
    y, _ = learn.get_preds(DatasetType.Test)
    y = torch.argmax(y, dim=1)
    preds = [learn.data.classes[int(x)] for x in y]
    rm -r 'Downloaded_Images'
    resultsFile = open('results.csv', 'wb')
    wr = csv.writer(resultsFile)
    wr.writerows([preds])
    return FileResponse('results.csv')

Now, tabular is a pretty special case. In the business world, most work will be done by sending in large chunks of data for analysis on a served model *somewhere*. The company may have it hooked into a GPU to account for the faster time, and having it shut off and on based on when the server gets a request. The models we export are generally CPU based models, but we can adjust for this if needed.

Now let's recreate what we did above for tabular data. Instead we will load it into pandas.

In [0]:
import StringIO
import csv

@app.route('/analyze', methods=['POST'])
async def analyze(request):
    data = await request.form()
    content = await (data['file'].read())
    s = str(content, 'utf-8')
    data = StringIO(s)
    df = pd.read_csv(data)
    data = TabularList.from_df(df, path='', cat_names = cat_names,
                              cont_names = cont_names, procs = procs)
    learn = load_learner(path, export_file_name, test=data)
    y, _ = learn.get_preds(DatasetType.Test)
    y = torch.argmax(y, dim=1)
    preds = [learn.data.classes[int(x)] for x in y]
    df['Predictions'] = preds
    
    path3 = Path('app/static/')
    df.to_csv(path3/'results.csv')
    
    return FileRespose('results.csv', media_type='csv')

We also need to adjust our JavaScript a little bit too. Specifically the end bit of the Anlyze function:

In [0]:
  xhr.onload = function(e) {
    if (this.readyState === 4) {
      el("result-label").innerHTML = `Result = Good`;
      
      download('results.csv', 'results.csv');
      xhr.send();
    }
    el("analyze-button").innerHTML = "Analyze";
  };

  var fileData = new FormData();
  fileData.append("file", uploadFiles[0]);
  xhr.send(fileData);
}

As we are now taking FormData